# CONVERTING THE EXCEL DATA INTO OUR REQUIRED FORM 

In [1]:
import pandas as pd
import os

# List of values to keep
values_to_keep = [
    "employerName",
    "employerAddressStreet_name",
    "employerAddressCity",
    "employerAddressState",
    "employerAddressZip",
    "einEmployerIdentificationNumber",
    "employeeName",
    "ssnOfEmployee",
    "box1WagesTipsAndOtherCompensations",
    "box2FederalIncomeTaxWithheld",
    "box3SocialSecurityWages",
    "box4SocialSecurityTaxWithheld",
    "box16StateWagesTips",
    "box17StateIncomeTax",
    "taxYear"
]

# Folder containing the files with .tsv extension but comma-separated content
folder_path = r'C:\Users\HP\Downloads\proj\boxes_transcripts_labels'  # Path to the folder
output_folder_path = r'C:\Users\HP\Downloads\proj\individual'  # Folder for output Excel files
extension = '.tsv'  # File extension

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Loop through each file in the specified folder
for file in os.listdir(folder_path):
    if file.endswith(extension):
        print(f"Processing file: {file}")  # Debugging statement
        
        # Combine the folder path and file name
        filePath = os.path.join(folder_path, file)
        
        # Read the file, treating it as a CSV (comma-separated file)
        df = pd.read_csv(filePath, sep=',', header=None)
        
        # Assign column names
        df.columns = ['start_index', 'end_index', 'x_top_left', 'y_top_left', 'x_bottom_right', 'y_bottom_right', 'transcript', 'field']
        
        # Filter the DataFrame to keep only rows where 'field' is in values_to_keep
        df_filtered = df[df["field"].isin(values_to_keep)]
        
        # Initialize a list to store grouped DataFrames
        grouped_data_frames = []

        # Group by 'field' and concatenate 'transcript'
        for field in values_to_keep:
            if field in df_filtered['field'].values:  # Check if field exists in the filtered DataFrame
                newDataFrame = df_filtered[df_filtered['field'] == field].groupby('field').agg({
                    'transcript': lambda x: ' '.join(x).strip()  # Concatenate and strip whitespace
                }).reset_index()
                
                # Strip spaces and handle any potential gaps in the data
                newDataFrame['transcript'] = newDataFrame['transcript'].apply(lambda x: x.replace(' ', '').strip())
                
                # Append the grouped DataFrame to the list
                grouped_data_frames.append(newDataFrame)

        # Create a new DataFrame if there are any grouped data
        if grouped_data_frames:  # Check if there's any data to concatenate
            df_combined = pd.concat(grouped_data_frames, ignore_index=True)

            # Create a new Excel file in the specified output folder
            excel_file_path = os.path.join(output_folder_path, f"{os.path.splitext(file)[0]}.xlsx")  # Use the same name as the TSV file

            # Write the combined DataFrame to a new Excel file
            df_combined.to_excel(excel_file_path, index=False)
            print(f"Excel file created: {excel_file_path}")  # Debugging statement
        else:
            print(f"No data to save for {file}.")  # Debugging statement

print("Processing complete.")


Processing file: number1.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number1.xlsx
Processing file: number10.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number10.xlsx
Processing file: number100.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number100.xlsx
Processing file: number101.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number101.xlsx
Processing file: number102.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number102.xlsx
Processing file: number103.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number103.xlsx
Processing file: number104.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number104.xlsx
Processing file: number105.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number105.xlsx
Processing file: number106.tsv
Excel file created: C:\Users\HP\Downloads\proj\individual\number106.xlsx
Processing file: number107.tsv
Excel file created: C:\Users\HP\Downloa

In [4]:
import pandas as pd
import os

# Define the list of columns to extract
columns = [
    "employerName", "employerAddressStreet_name", "employerAddressCity", 
    "employerAddressState", "employerAddressZip", "einEmployerIdentificationNumber", 
    "employeeName", "ssnOfEmployee", "box1WagesTipsAndOtherCompensations", 
    "box2FederalIncomeTaxWithheld", "box3SocialSecurityWages", 
    "box4SocialSecurityTaxWithheld", "box16StateWagesTips", 
    "box17StateIncomeTax", "taxYear"
]

# Initialize an empty DataFrame to store results
result_df = pd.DataFrame(columns=columns)

# Path where your Excel files are located
path = r'C:/Users/HP/Downloads/proj/individual'

# Loop through all Excel files in the specified directory
for filename in os.listdir(path):
    if filename.endswith('.xlsx'):
        # Read the Excel file
        file_path = os.path.join(path, filename)
        df = pd.read_excel(file_path)

        # Initialize a new row for the new employee (for each file)
        new_row = {col: None for col in columns}  # Initialize with None for all columns
        
        # Iterate row-wise in the current file
        for _, row in df.iterrows():
            currField = row['field']
            currTranscript = row['transcript']
            
            # Clean data by stripping any leading/trailing spaces only if the transcript is a string
            if isinstance(currTranscript, str):
                currTranscript = currTranscript.strip()

            # Set the corresponding field value for the current file's row
            if currField in columns:
                new_row[currField] = currTranscript
        
        # Convert the new_row dictionary into a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new_row_df with result_df
        result_df = pd.concat([result_df, new_row_df], ignore_index=True)
        
# Convert numeric columns to float (if needed) after processing all files
numeric_columns = [
    "box1WagesTipsAndOtherCompensations", "box2FederalIncomeTaxWithheld", 
    "box3SocialSecurityWages", "box4SocialSecurityTaxWithheld", 
    "box16StateWagesTips", "box17StateIncomeTax"
]

# Clean the numeric data before converting (remove unwanted characters)
for col in numeric_columns:
    result_df[col] = result_df[col].replace({'\$': '', ',': '', ' ': ''}, regex=True)  # Strip spaces, commas, and dollar signs
    result_df[col] = pd.to_numeric(result_df[col], errors='coerce')  # Convert to numeric, handle errors gracefully

# Save the resulting DataFrame to an Excel file
output_file_path = r'C:/Users/HP/Downloads/proj/individual/result.xlsx'
result_df.to_excel(output_file_path, index=False)

print(f"Processing complete. File saved to {output_file_path}.")


Processing complete. File saved to C:/Users/HP/Downloads/proj/individual/result.xlsx.
